In [62]:
import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv('../MNH TTO Data/MNH TTO Data - MERGED.csv', na_values=' ')

In [64]:
df

,Date,Bed no.,Case no.,Can close,No. of drugs,Time TTO received (24 hr format) (eg 1340),Timing of Packing (24 hr format / PBH) (eg 1340 / PBH),Timing of Dispensing (24 hr format),Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),...,Timing of Dispensing,20 min and below,Between 20 to 30 min,30min and below,More than 30 min,Weekday,Weekday Count,Weekend,Weekend Count,NOT Ward 9
0,01-01-21,905,4020095917,839,4,828,pbh,840,NaN,PBH,...,8:40 AM,1,0,1,0,TRUE,1,FALSE,0,0
1,01-01-21,1202,4020097017,934,8,904,850,NaN,wait for 2nd rx,##############################################...,...,#VALUE!,0,0,0,0,TRUE,1,FALSE,0,1
2,01-01-21,1224,4020095632,934,1,901,905,920,NaN,0:04,...,9:20 AM,1,0,1,0,TRUE,1,FALSE,0,1
3,01-01-21,818,4020097033,934,1,900,901,915,NaN,0:01,...,9:15 AM,1,0,1,0,TRUE,1,FALSE,0,1
4,01-01-21,803,4020097026,1029,1,908,915,1027,"can close, added on metformin and glipizide. 1...",0:07,...,10:27 AM,0,0,0,1,TRUE,1,FALSE,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1311,1,1300,1300,1310,add on TTO,0:00,...,1:10 PM,1,0,1,0,TRUE,1,FALSE,0,1
15518,31-12-21,1205,4021107518,1351,1,1335,pbh,1347,NaN,PBH,...,1:47 PM,1,0,1,0,TRUE,1,FALSE,0,1
15519,31-12-21,1131,4021107644,1426,3,1408,1417,1434,NaN,0:09,...,2:34 PM,0,1,1,0,TRUE,1,FALSE,0,1
15520,31-12-21,622,4021108689,1446,2,1359,1409,1441,1431 pt in bathroom,0:10,...,2:41 PM,0,0,0,1,TRUE,1,FALSE,0,1


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15522 entries, 0 to 15521
Data columns (total 23 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   Date                                                    15521 non-null  object
 1   Bed no.                                                 15512 non-null  object
 2   Case no.                                                15516 non-null  object
 3   Can close                                               15454 non-null  object
 4   No. of drugs                                            15520 non-null  object
 5   Time TTO received (24 hr format) (eg 1340)              15517 non-null  object
 6   Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)  15508 non-null  object
 7   Timing of Dispensing (24 hr format)                     15512 non-null  object
 8   Comments (for delayed cases) (eg. Pt sleeping 

# Drop columns that should be ignore

In [66]:
# drop repeated column
df.drop(['Can close', 'Time TTO received (24 hr format) (eg 1340)', 'Timing of Packing (24 hr format / PBH) (eg 1340 / PBH)', 'Timing of Dispensing (24 hr format)',
        'Weekday', 'Weekday Count', 'Weekend', 'Weekend Count', 'NOT Ward 9'], axis=1, inplace=True)

In [67]:
#df.drop(df.iloc[:, 18:23], inplace = True, axis = 1)

# Rename common columns

In [68]:
#rename columns
df.rename({'Case no.': 'Case Number', 
            'Bed no.': 'Bed Number', 
            'No. of drugs': 'No. of Drugs',
           'Time TTO received': 'Time TTO Received',
            'Timing of packing': 'Time TTO Packing', 
            'Timing of Dispensing': 'Time TTO Dispensed'},   
           axis=1, inplace=True)

# Handle missing values

In [69]:
df.isnull().sum()

Date                                                      1
Bed Number                                               10
Case Number                                               6
No. of Drugs                                              2
Comments (for delayed cases) (eg. Pt sleeping etc)    11665
Time Taken To Pack (mins)                                 1
Time Taken to Reach Pt (mins)                             0
Time TTO Received                                         0
Time TTO Packing                                          0
Time TTO Dispensed                                        0
20 min and below                                          0
Between 20 to 30 min                                      0
30min and below                                           0
More than 30 min                                          0
dtype: int64

In [70]:
#df.iloc[5750:5770]

### Handle missing date

In [71]:
missing_date = df[df['Date'].isnull()]
missing_date

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
8349,NaN,911,4021053861,2,NaN,0:11,0:28,7:54 AM,8:05 AM,8:22 AM,0,1,1,0


In [72]:
# fill up with 15-07-21 based on the given data CSV
df[['Date']] = df[['Date']].fillna('15-07-21')

### Handle missing Time TTO received

In [73]:
# drop rows with missing Time TTO received -> 1 row only
df.replace('#VALUE!', np.nan, inplace = True)
df = df.dropna(subset=['Time TTO Received'])
#df.replace('blank', np.nan, inplace = True)

### Handle missing bed number

In [74]:
# handle those with missing bed number, case no by replacing with blank
#missing_bed_no = df[df['Bed Number'].isnull()]
df[['Bed Number']] = df[['Bed Number']].fillna('NA')

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Handle missing case number

In [75]:
df['Case Number'] = df['Case Number'].replace('  ', np.nan)
df[['Case Number']] = df[['Case Number']].fillna('NA')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Handle missing Time Taken To Pack (mins)

In [76]:
# handle row with missing Time Taken To Pack (mins) by filling up with PBH 
df[df['Time Taken To Pack (mins)'].isnull()]

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
10040,25-08-21,1105,4021069998,1,NaN,NaN,0:17,9:39 AM,PBH,9:56 AM,1,0,1,0


In [77]:
df[['Time Taken To Pack (mins)']] = df[['Time Taken To Pack (mins)']].fillna('PBH')

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Handle missing Time TTO packing

In [78]:
df[df['Time TTO Packing'].isnull()].head(3)

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
506,14-01-21,819,4021001825,1,NaN,blank,0:12,9:48 AM,NaN,10:00 AM,1,0,1,0
885,22-01-21,203,4021005352,6,NaN,blank,blank,3:54 PM,NaN,NaN,0,0,0,0
1324,02-02-21,cot 926,4021008672,1,845 dr in room,blank,2:09,7:26 AM,NaN,9:35 AM,0,0,0,1


In [79]:
df[['Time TTO Packing']] = df[['Time TTO Packing']].fillna('NA')

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


### Handle missing Time TTO Dispensed

In [80]:
df[df['Time TTO Dispensed'].isnull()].head(3)

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
1,01-01-21,1202,4020097017,8,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,NaN,0,0,0,0
885,22-01-21,203,4021005352,6,NaN,blank,blank,3:54 PM,NA,NaN,0,0,0,0
2462,02-03-21,1209,4021015997,4,NaN,0:10,blank,9:30 AM,9:40 AM,NaN,0,0,0,0


In [81]:
# handle row with missing Time TTO packing, Time TTO dispensing with blank because it is empty but dont want to drop
df[['Time TTO Dispensed']] = df[['Time TTO Dispensed']].fillna('NA')
#df[df['Time TTO packing'].isnull()].head()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [82]:
df.isnull().sum()

Date                                                      0
Bed Number                                                0
Case Number                                               0
No. of Drugs                                              0
Comments (for delayed cases) (eg. Pt sleeping etc)    11659
Time Taken To Pack (mins)                                 0
Time Taken to Reach Pt (mins)                             0
Time TTO Received                                         0
Time TTO Packing                                          0
Time TTO Dispensed                                        0
20 min and below                                          0
Between 20 to 30 min                                      0
30min and below                                           0
More than 30 min                                          0
dtype: int64

In [83]:
df

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
0,01-01-21,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM,1,0,1,0
1,01-01-21,1202,4020097017,8,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,NA,0,0,0,0
2,01-01-21,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM,1,0,1,0
3,01-01-21,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM,1,0,1,0
4,01-01-21,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,31-12-21,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM,1,0,1,0
15518,31-12-21,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM,1,0,1,0
15519,31-12-21,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM,0,1,1,0
15520,31-12-21,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM,0,0,0,1


In [84]:
#no need to handle missing values for comments since they can leave comments or not

# Handle rows with special characters

### Handle non numeric value in No. of drugs

In [85]:
#select rows not numeric
df[df['No. of Drugs'].apply(lambda x: not x.isnumeric())]

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
952,24-01-21,1012,4021006560,-,patient rejected meds,0:34,0:59,10:36 AM,11:10 AM,11:35 AM,0,0,0,1
3560,26-03-21,COT928,4021023823,relabeling,NaN,blank,1:13,7:41 AM,NA,8:54 AM,0,0,0,1


In [86]:
# replace with Unknown number 
df['No. of Drugs'] = df['No. of Drugs'].str.replace('-','NA')
df['No. of Drugs'] = df['No. of Drugs'].str.replace('relabeling','NA')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Handle #### value in Time Taken To Pack (mins) / Time Taken to Reach Pt (mins)

In [87]:
df_special = df.loc[df['Time Taken To Pack (mins)'].str.contains(r'#') == True]
df_special.head()

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
1,01-01-21,1202,4020097017,8,wait for 2nd rx,##############################################...,blank,9:04 AM,8:50 AM,NA,0,0,0,0
284,09-01-21,626,4020093087,2,NaN,##############################################...,0:40,9:40 AM,8:48 AM,10:20 AM,0,0,0,1
341,10-01-21,p08,4021002120,3,NaN,##############################################...,0:18,12:13 PM,11:54 AM,12:31 PM,1,0,1,0
345,10-01-21,1214,4020096026,8,NaN,##############################################...,0:43,2:09 PM,2:02 PM,2:52 PM,0,0,0,1
474,13-01-21,P05,4021001516,9,"long TTO, CD",##############################################...,##############################################...,3:40 PM,2:59 PM,3:34 PM,1,0,1,0


In [88]:
df['Time Taken To Pack (mins)'] = df['Time Taken To Pack (mins)'].str.replace('###############################################################################################################################################################################################################################################################', 
                                                                              'PBH') # replace with PBH

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [89]:
# replace blank with NA
df['Time Taken To Pack (mins)'] = df['Time Taken To Pack (mins)'].str.replace('blank', 'NA')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [90]:
df['Time Taken to Reach Pt (mins)'] = df['Time Taken to Reach Pt (mins)'].str.replace('###############################################################################################################################################################################################################################################################', 
                                                                              'PBH') # replace with PBH

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [91]:
# replace blank with NA
df['Time Taken to Reach Pt (mins)']  = df['Time Taken to Reach Pt (mins)'].str.replace('blank', 'NA')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
df.iloc[474:476]

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
474,13-01-21,P05,4021001516,9,"long TTO, CD",PBH,PBH,3:40 PM,2:59 PM,3:34 PM,1,0,1,0
475,13-01-21,606,4021003450,1,1455 pt in ot,0:17,1:26,2:34 PM,2:51 PM,4:00 PM,0,0,0,1


# Replace year not equal to 2021 as 2021 in Date

In [93]:
# change to datetime formate
df['Date'] = pd.to_datetime(df['Date'])

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [94]:
df_filtered = df[df['Date'].dt.strftime('%Y') != '2021']
df_filtered.count()

Date                                                  47
Bed Number                                            47
Case Number                                           47
No. of Drugs                                          47
Comments (for delayed cases) (eg. Pt sleeping etc)    14
Time Taken To Pack (mins)                             47
Time Taken to Reach Pt (mins)                         47
Time TTO Received                                     47
Time TTO Packing                                      47
Time TTO Dispensed                                    47
20 min and below                                      47
Between 20 to 30 min                                  47
30min and below                                       47
More than 30 min                                      47
dtype: int64

In [95]:
df_filtered

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
8626,2022-07-22,824,4021055919,11,NaN,PBH,0:17,8:53 AM,PBH,9:10 AM,1,0,1,0
8627,2023-07-22,612,4021059316,4,paeds relabel,0:13,0:49,8:21 AM,8:34 AM,9:10 AM,0,0,0,1
8628,2024-07-22,920,4021035651,2,NaN,PBH,0:27,8:44 AM,PBH,9:11 AM,0,1,1,0
8629,2025-07-22,921,4021050039,3,NaN,0:14,0:42,8:32 AM,8:46 AM,9:14 AM,0,0,0,1
8630,2026-07-22,1020,4021059853,6,NaN,0:35,0:54,8:36 AM,9:11 AM,9:30 AM,0,0,0,1
8631,2027-07-22,214,4021059893,1,NaN,PBH,0:41,9:17 AM,PBH,9:58 AM,0,0,0,1
8632,2028-07-22,817,4021059498,1,NaN,0:11,0:20,9:15 AM,9:26 AM,9:35 AM,1,0,1,0
8633,2029-07-22,814,4021059839,5,NaN,PBH,0:31,9:09 AM,PBH,9:40 AM,0,1,1,0
8634,2030-07-22,1022,4021058893,2,NaN,0:21,0:41,8:49 AM,9:10 AM,9:30 AM,0,0,0,1
8635,2031-07-22,616,4021058545,6,awiting stocks,PBH,1:08,8:52 AM,PBH,10:00 AM,0,0,0,1


In [96]:
df.loc[8626:8673, 'Date'] = '2021-07-22'
df['Date'] = pd.to_datetime(df['Date']) # change back to datetime format from obj

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15513 entries, 0 to 15521
Data columns (total 14 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   Date                                                15513 non-null  datetime64[ns]
 1   Bed Number                                          15513 non-null  object        
 2   Case Number                                         15513 non-null  object        
 3   No. of Drugs                                        15513 non-null  object        
 4   Comments (for delayed cases) (eg. Pt sleeping etc)  3854 non-null   object        
 5   Time Taken To Pack (mins)                           15513 non-null  object        
 6   Time Taken to Reach Pt (mins)                       15513 non-null  object        
 7   Time TTO Received                                   15513 non-null  object        
 8   Time T

In [98]:
df.loc[8619:8669]

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min
8619,2021-07-21,215,4021060153,2,NaN,0:06,0:33,4:53 PM,4:59 PM,5:26 PM,0,0,0,1
8620,2021-07-21,613,4021059880,3,can go at 1720,0:04,1:15,4:16 PM,4:20 PM,5:31 PM,0,0,0,1
8621,2021-07-21,468,4021060222,1,NaN,0:08,0:17,5:37 PM,5:45 PM,5:54 PM,1,0,1,0
8622,2021-07-21,604,4021059617,4,1729 can go,0:20,4:05,1:50 PM,2:10 PM,5:55 PM,0,0,0,1
8623,2021-07-21,1006,4021060165,2,NaN,0:06,0:17,6:43 PM,6:49 PM,7:00 PM,1,0,1,0
8624,2021-07-21,p03,4021060107,4,NaN,0:00,0:19,7:22 PM,7:22 PM,7:41 PM,1,0,1,0
8625,2021-07-22,912,4021051742,5,NaN,0:09,0:35,8:15 AM,8:24 AM,8:50 AM,0,0,0,1
8626,2021-07-22,824,4021055919,11,NaN,PBH,0:17,8:53 AM,PBH,9:10 AM,1,0,1,0
8627,2021-07-22,612,4021059316,4,paeds relabel,0:13,0:49,8:21 AM,8:34 AM,9:10 AM,0,0,0,1
8628,2021-07-22,920,4021035651,2,NaN,PBH,0:27,8:44 AM,PBH,9:11 AM,0,1,1,0


# Get day column from date

In [99]:
df['Weekday'] = df['Date'].dt.day_name()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [100]:
df

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min,Weekday
0,2021-01-01,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM,1,0,1,0,Friday
1,2021-01-01,1202,4020097017,8,wait for 2nd rx,PBH,NA,9:04 AM,8:50 AM,NA,0,0,0,0,Friday
2,2021-01-01,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM,1,0,1,0,Friday
3,2021-01-01,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM,1,0,1,0,Friday
4,2021-01-01,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM,0,0,0,1,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,2021-12-31,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM,1,0,1,0,Friday
15518,2021-12-31,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM,1,0,1,0,Friday
15519,2021-12-31,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM,0,1,1,0,Friday
15520,2021-12-31,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM,0,0,0,1,Friday


# Add additional metrics

### Get the minutes from Time Taken To Pack (mins) & Time Taken to Reach Pt (mins)

In [101]:
#df_filtered = df[df['Time Taken to Reach Pt (mins)'] == 'PBH']
#df_filtered

In [109]:
tto_received = pd.to_datetime(df['Time TTO Received'].astype(str)) 
tto_dispensed = pd.to_datetime(df[df['Time Taken to Reach Pt (mins)'] != 'NA']['Time TTO Dispensed'].astype(str))

#df[df['Time Taken to Reach Pt (mins)'] == 'PBH']
df['Overall Time Taken to Dispensed'] = tto_dispensed.sub(tto_received).dt.total_seconds().div(60)
df

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min,Weekday,Overall Time Taken to Dispensed
0,2021-01-01,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM,1,0,1,0,Friday,12.0
1,2021-01-01,1202,4020097017,8,wait for 2nd rx,PBH,NA,9:04 AM,8:50 AM,NA,0,0,0,0,Friday,NaN
2,2021-01-01,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM,1,0,1,0,Friday,19.0
3,2021-01-01,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM,1,0,1,0,Friday,15.0
4,2021-01-01,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM,0,0,0,1,Friday,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,2021-12-31,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM,1,0,1,0,Friday,10.0
15518,2021-12-31,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM,1,0,1,0,Friday,12.0
15519,2021-12-31,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM,0,1,1,0,Friday,26.0
15520,2021-12-31,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM,0,0,0,1,Friday,42.0


In [110]:
#df[df['Time Taken to Reach Pt (mins)'] != 'NA']['Time TTO Dispensed']

### Add whether meet KPI column

In [111]:
def f(row):
    if row['Overall Time Taken to Dispensed'] > 45:
        val = 0
    elif row['Overall Time Taken to Dispensed'] < 0:
        val = 'NA'
    else:
        val = 1
    return val

In [112]:
df['Meet KPI'] = df.apply(f, axis=1)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [113]:
df

,Date,Bed Number,Case Number,No. of Drugs,Comments (for delayed cases) (eg. Pt sleeping etc),Time Taken To Pack (mins),Time Taken to Reach Pt (mins),Time TTO Received,Time TTO Packing,Time TTO Dispensed,20 min and below,Between 20 to 30 min,30min and below,More than 30 min,Weekday,Overall Time Taken to Dispensed,Meet KPI
0,2021-01-01,905,4020095917,4,NaN,PBH,0:12,8:28 AM,PBH,8:40 AM,1,0,1,0,Friday,12.0,1
1,2021-01-01,1202,4020097017,8,wait for 2nd rx,PBH,NA,9:04 AM,8:50 AM,NA,0,0,0,0,Friday,NaN,1
2,2021-01-01,1224,4020095632,1,NaN,0:04,0:19,9:01 AM,9:05 AM,9:20 AM,1,0,1,0,Friday,19.0,1
3,2021-01-01,818,4020097033,1,NaN,0:01,0:15,9:00 AM,9:01 AM,9:15 AM,1,0,1,0,Friday,15.0,1
4,2021-01-01,803,4020097026,1,"can close, added on metformin and glipizide. 1...",0:07,1:19,9:08 AM,9:15 AM,10:27 AM,0,0,0,1,Friday,79.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15517,2021-12-31,824,4021107656,1,add on TTO,0:00,0:10,1:00 PM,1:00 PM,1:10 PM,1,0,1,0,Friday,10.0,1
15518,2021-12-31,1205,4021107518,1,NaN,PBH,0:12,1:35 PM,PBH,1:47 PM,1,0,1,0,Friday,12.0,1
15519,2021-12-31,1131,4021107644,3,NaN,0:09,0:26,2:08 PM,2:17 PM,2:34 PM,0,1,1,0,Friday,26.0,1
15520,2021-12-31,622,4021108689,2,1431 pt in bathroom,0:10,0:42,1:59 PM,2:09 PM,2:41 PM,0,0,0,1,Friday,42.0,1


In [114]:
df.to_csv("MNH TTO TAT (cleaned).csv", index = None, header=True)

# EDA

In [ ]:
df = pd.read_csv('MNH TTO TAT (cleaned).csv')

In [ ]:
df.describe()